In [1]:
!pip install -q -U google-generativeai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.0 requires protobuf==3.20.3, but you have protobuf 5.29.5 which is incompatible.
onnx 1.13.1rc2 requires protobuf<4,>=3.20.2, but you have protobuf 5.29.5 which is incompatible.
wandb 0.16.6 requires protobuf!=4.21.0,<5,>=3.19.0; python_version > "3.9" and sys_platform == "linux", but you have protobuf 5.29.5 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [57]:
import pandas as pd
import google.generativeai as genai
import os

In [47]:
def generate_report(feature_dict):

    text = prompt.format(feature_dict)
    response = model.generate_content(text)
    return response

In [48]:
#input your gemini api key 
api_key = ''

In [49]:
prompt = 'Think as if you are a radiologist, you have a table/dictionary for lung nodule evaluation, and you want to transform it into radiology reports. Here is the dictionary {}. Just show me the report. Do not add any additional information. In English! You should have an impression and findings part. Findings are bullet points listing each of the features separately. These features should be in random order. Combine nodule margins and additional nodule margins. If there is a missing value, just ignore it. For impression, do a quick summary of the findings in sentence and then state the suspiciousness of lung cancer. You can mention the absences in the findings but do not mention those in the impression.'
print(prompt)

Think as if you are a radiologist, you have a table/dictionary for lung nodule evaluation, and you want to transform it into radiology reports. Here is the dictionary {}. Just show me the report. Do not add any additional information. In English! You should have an impression and findings part. Findings are bullet points listing each of the features separately. These features should be in random order. Combine nodule margins and additional nodule margins. If there is a missing value, just ignore it. For impression, do a quick summary of the findings in sentence and then state the suspiciousness of lung cancer. You can mention the absences in the findings but do not mention those in the impression.


In [50]:
genai.configure(api_key = api_key)
model = genai.GenerativeModel('gemini-1.5-flash')

In [51]:
df = pd.read_csv('../dataset_csv/sample_csv_with_semantic_feats.csv')
cols = ['longest_axial_diameter_(mm)', 'short_diameter_(mm)', 'mean_diameter','psn_longest_diameter_solid_component',
'nodule_margin_conspicuity','nodule_margins', 'additional_nodule_margins', 'nodule_shape', 'nodule_consistency',
    'nodule_reticulation', 'cyst-like_spaces','intra-nodular_bronchiectasis', 'necrosis', 'cavitation',
   'eccentric_calcification', 'airway_cut-off', 'pleural_attachment','pleural_retraction', 'vascular_convergence', 'septal_stretching',
   'paracicatricial_emphysema','level_of_suspicion_of_lung_cancer']

In [58]:
save_dir = '../report_generation'
for i in range(len(df)):
    save_name = str(df.iloc[i].pid)+'_'+str(df.iloc[i].nodule_id)+'.txt'
    save_path = os.path.join(save_dir, save_name)
    feature_dict = df[cols].iloc[0].to_dict()
    report = generate_report(feature_dict)
    report = report.text.replace('#','').replace('*','').replace('-',' ').replace('/',', ')
    with open(save_path, "w") as file:
        file.write(report)